# Plots

In [1]:
# Import to be able to import python package from src
import sys
sys.path.insert(0, '../src')

In [2]:
import pandas as pd
import ontime as on
from darts.datasets import EnergyDataset

## Load data

In [3]:
ts = EnergyDataset().load()

Complete TimeSeries

In [4]:
df = ts.pd_dataframe()
df = df.interpolate()
cols = ['generation biomass', 'generation solar', 'generation nuclear']
df = df[cols]

In [5]:
ts = on.TimeSeries.from_dataframe(df)

Prepare data

In [6]:
ts_uni = ts['generation solar'].slice(pd.Timestamp('2015'), pd.Timestamp('2016'))
ts_multi = ts.slice(pd.Timestamp('2015'), pd.Timestamp('2016'))

## Primitive Plots

### Line(s)

With univariate TimeSeries

In [7]:
on.Plot(ts_uni.head(400))\
    .add(on.marks.line)\
    .properties(width=600)\
    .show()

alt.LayerChart(...)

with multivariate TimeSeries

In [8]:
on.Plot(ts_multi.head(400))\
    .add(on.marks.line)\
    .properties(width=600)\
    .show()

alt.LayerChart(...)

### Dots

With univariate TimeSeries

In [9]:
on.Plot(ts_uni.head(400))\
    .add(on.marks.dots)\
    .properties(width=600)\
    .show()

alt.LayerChart(...)

with multivariate TimeSeries

In [10]:
on.Plot(ts_multi.head(400))\
    .add(on.marks.dots)\
    .properties(width=600)\
    .show()

alt.LayerChart(...)

### Areas

With a single time series

In [11]:
on.Plot(ts_uni.head(400))\
    .add(on.marks.area)\
    .properties(width=600)\
    .show()

alt.LayerChart(...)

With a multivariate time series it works with exactly two

In [12]:
from darts import concatenate

In [13]:
# First we create the series with two components
ts_ci = concatenate([
    ts_multi.univariate_component(0), 
    ts_multi.univariate_component(1)
], axis=1)
ts_ci = on.TimeSeries.from_darts(ts_ci)

In [14]:
# Then we plot it
on.Plot(ts_ci.head(200))\
    .add(on.marks.area, title='Diff. between solar and biomass generation')\
    .properties(width=600, height=200)\
    .show()

alt.LayerChart(...)

### Heatmaps

with univariate TimeSeries

In [15]:
on.Plot(ts_uni.head(1000))\
    .add(on.marks.heatmap)\
    .properties(width=600, height=50)\
    .show()

alt.LayerChart(...)

with multivariate Heatmap

In [16]:
on.Plot(ts_multi.head(1000))\
    .add(on.marks.heatmap)\
    .properties(width=600, height=150)\
    .show()

alt.LayerChart(...)

---

## Combined Plots

Most of the plots in onTime can be combined as they are based on Altair layered charts. For instance, you can do the following to have a dots on a line.

In [17]:
on.Plot()\
    .add(on.marks.dots, ts_multi.univariate_component(1).head(400))\
    .add(on.marks.line, ts_multi.univariate_component(0).head(400))\
    .properties(width=600, height=200)\
    .show()

alt.LayerChart(...)

---

## Thematic Plots

### Forecasts

In [18]:
ts_train, ts_test = ts_uni.split_before(0.9)

In [19]:
from ontime.context import common

In [20]:
model = common.GenericPredictor()
model.fit(ts_train)

In [21]:
ts_pred = model.predict(24 * 3)

In [22]:
ts_train = ts_train.rename({'generation solar':'Training set'})
ts_test = ts_test.rename({'generation solar':'Test set'})
ts_pred = ts_pred.rename({'generation solar':'Forecast'})

Plot a prediction

In [23]:
(
on.Plot()
    .add(on.marks.line, ts_test.head(24 * 3), type='dashed')
    .add(on.marks.line, ts_train.tail(24 * 4))
    .add(on.marks.line, ts_pred)
    .properties(width=600, height=200)
    .show()
)

alt.LayerChart(...)

### Anomalies

Create the mock data

In [24]:
td_point = on.detectors.quantile(high_quantile=0.99)
td_collective = on.detectors.threshold(low_threshold=-30)
td_contextual = on.detectors.quantile(high_quantile=0.98)

Add anomalies

In [25]:
import numpy as np
import random

def add_point_anomalies(ts, n, value):
    df = ts.pd_dataframe()
    random_indices = np.random.choice(df.index, size=n, replace=False)
    df.loc[random_indices] = value
    return on.TimeSeries.from_dataframe(df)

def add_collective_anomalies(ts, n, min_duration=10, max_duration=20):
    df = ts.pd_dataframe()
    for i in range(n+1):
        block_duration = random.randint(min_duration, max_duration)
        start_index = np.random.choice(df.index[:-block_duration])
        end_index = start_index + pd.Timedelta(days=block_duration - 1)
        df.loc[start_index:end_index] = -40
    return on.TimeSeries.from_dataframe(df)

Select univariate component

In [26]:
ts = ts.univariate_component(0)

In [27]:
ts = add_point_anomalies(ts, 10, 30)
ts = add_collective_anomalies(ts, 4)

Create binary time series

In [28]:
td_point.fit(ts)
td_contextual.fit(ts)

ts_ano_point = td_point.detect(ts)
ts_ano_collective = td_collective.detect(ts)
ts_ano_contextual = td_contextual.detect(ts)

In [29]:
ts_ano_point.univariate_component(0)

<TimeSeries (DataArray) (time: 35064, component: 1, sample: 1)> Size: 281kB
array([[[0.]],

       [[0.]],

       [[0.]],

       ...,

       [[0.]],

       [[0.]],

       [[0.]]])
Coordinates:
  * time       (time) datetime64[ns] 281kB 2014-12-31T23:00:00 ... 2018-12-31...
  * component  (component) object 8B 'generation biomass'
Dimensions without coordinates: sample
Attributes:
    static_covariates:  None
    hierarchy:          None

In [30]:
col = ts_ano_point.columns[0]
ts_ano_point = ts_ano_point.rename({col: 'Ponctual anomalies'})
ts_ano_collective = ts_ano_collective.rename({col: 'Collective anomalies'})
ts_ano_contextual = ts_ano_contextual.rename({col: 'Contextual anomalies'})

Plot the time series with marked anomalies

In [31]:
# Define windows for plotting
start = 24 * 7 * 4 * 9
duration = 24 * 7 * 10
end = start + duration

In [32]:
# Actually plot
(
on.Plot(ts[start:end])
    .add(on.marks.mark, data=ts_ano_contextual[start:end], type='highlight')
    .add(on.marks.mark, data=ts_ano_collective[start:end], type='background')
    .add(on.marks.line)
    .add(on.marks.mark, data=ts_ano_point[start:end], type='dot')
    .properties(width=800, height=200)
    .show()
)

alt.LayerChart(...)

### Confidence Intervals

In [33]:
# Generate two time series
ts1 = on.generators.random_walk().generate(start=pd.Timestamp('2022-01-01'), end=pd.Timestamp('2022-12-31'))
ts2 = on.generators.random_walk().generate(start=pd.Timestamp('2022-01-01'), end=pd.Timestamp('2022-12-31'))

In [34]:
# First we create the series with two components
ts1_abs = ts1.map(np.abs)
ts2_abs = ts2.map(np.abs)
ts_ci = concatenate([ts1_abs, ts2_abs], axis=1)
ts_ci = on.TimeSeries.from_darts(ts_ci)
ts_ci = ts_ci.rename({'random_walk': 'CI Upper bound', 'random_walk_1': 'CI Lower bound'})

# Then the hypothetical measurement
ts_mid = (ts1_abs + ts2_abs) / 2
ts_mid = ts_mid.rename({'random_walk': 'Measurement'})

In [35]:
# Then we plot it
(
on.Plot() # main line
    .add(on.marks.area, ts_ci.head(200), title='Confidence interval')
    .add(on.marks.line, ts_mid.head(200))
    .properties(width=600, height=200)
    .show()
)

alt.LayerChart(...)

---